# Numpy basics

As a reminder, objects in Python (including numbers!) are abstract and dynamic:

   * you don't know where they are in memory (pointer address); could be anywhere
   * you don't know how they're represented in terms of bytes
   * data types, member data, function arguments, etc. are checked at the last possible moment

And so they are slow.

In [1]:
import random
data = []
for i in range(1000000):
    data.append(random.gauss(0, 1))

In [2]:
%%timeit
data2 = []
for x in data:
    data2.append(x**2)

1 loop, best of 3: 147 ms per loop


But Numpy isn't.

In [3]:
import numpy
data = numpy.random.normal(0, 1, 1000000)

In [5]:
%%timeit
data2 = data**2

1000 loops, best of 3: 601 µs per loop


**How does it work?**

A Numpy array is everything a list of Python objects is not:

   * the data are known to be contiguous in memory (sequential access is important!)
   * you can directly access and manipulate their bytes
   * the data type of an array is specified once for the whole array
   * *bonus:* most methods benefit from hardware vectorization
   * *bonus:* all methods release Python's interpreter lock, so parallel threads can run at the same time
   * *bonus:* without a type pointer for each object, numbers use less memory

Numpy encourages a different order of operations: instead of processing a table of data one event at a time, it only helps if you process one operation (for all events) at a time.

In [6]:
px = numpy.random.normal(0, 30, 100000)
py = numpy.random.normal(0, 30, 100000)
pz = numpy.random.normal(0, 300, 100000)

Instead of

In [7]:
%%timeit
p = numpy.empty(100000)
for i in range(len(p)):                                   # for each px[i], py[i], pz[i]
    p[i] = numpy.sqrt(px[i]**2 + py[i]**2 + pz[i]**2)     # compute p[i]

1 loop, best of 3: 227 ms per loop


do

In [8]:
%%timeit
p = numpy.sqrt(px**2 + py**2 + pz**2)       # compute all px**2, then all py**2, then all pz**2, then sum all, then sqrt all

1000 loops, best of 3: 535 µs per loop


Normal math functions are *scalar* (e.g. binary operators like `+` or functions from `import math`). They perform one operation each time they appear in Python.

Numpy math functions are *vectorized.* Given equal-length arrays as input, they return the same length array as output, performing all loops in compiled C or even vectorized across a CPU. (Some implementations perform the work in parallel or on a GPU, but not the default one.)

In [9]:
small_array = numpy.array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9])

In [10]:
small_array**2

array([ 0,  1,  4,  9, 16, 25, 36, 49, 64, 81])

In [11]:
numpy.sqrt(small_array)

array([0.        , 1.        , 1.41421356, 1.73205081, 2.        ,
       2.23606798, 2.44948974, 2.64575131, 2.82842712, 3.        ])

In [15]:
import math
math.sqrt(small_array)

TypeError: only size-1 arrays can be converted to Python scalars

They are raw bytes; you can do whatever you want with them.

In [16]:
asbytes = small_array.view(numpy.uint8)
asbytes

array([0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0,
       0, 0, 3, 0, 0, 0, 0, 0, 0, 0, 4, 0, 0, 0, 0, 0, 0, 0, 5, 0, 0, 0,
       0, 0, 0, 0, 6, 0, 0, 0, 0, 0, 0, 0, 7, 0, 0, 0, 0, 0, 0, 0, 8, 0,
       0, 0, 0, 0, 0, 0, 9, 0, 0, 0, 0, 0, 0, 0], dtype=uint8)

In [17]:
asbytes[20] = 123
small_array

array([           0,            1, 528280977410,            3,
                  4,            5,            6,            7,
                  8,            9])

They may have arbitrarily many dimensions. Changing the dimensions (in a way that keeps the total number of items constant) *does not change the underlying data.*

In [23]:
small_array.reshape(5, 2)

array([[           0,            1],
       [528280977410,            3],
       [           4,            5],
       [           6,            7],
       [           8,            9]])

The columns can be named, making it easy to swap array-of-structs with struct-of-arrays. (A `recarray` is literally the same as 

In [25]:
recarray = small_array.view([("one", int), ("two", int)])
recarray

array([(           0, 1), (528280977410, 3), (           4, 5),
       (           6, 7), (           8, 9)],
      dtype=[('one', '<i8'), ('two', '<i8')])

In [26]:
recarray["one"]

array([           0, 528280977410,            4,            6,
                  8])

In [27]:
recarray["two"]

array([1, 3, 5, 7, 9])

In [29]:
recarray[2]

(4, 5)

Numpy arrays follow the same "slicing" rules as Python lists, though slicing becomes more important because it's much faster than iterating.

In [30]:
small_array[4:-2]

array([4, 5, 6, 7])

But they also have new rules, such as masking by an array of booleans:

In [31]:
mask = numpy.array([True, True, False, False, False, True, False, True, False, False])
small_array[mask]

array([0, 1, 5, 7])

In [32]:
small_array[mask] = numpy.array([1000, 1001, 1005, 1007])
small_array

array([        1000,         1001, 528280977410,            3,
                  4,         1005,            6,         1007,
                  8,            9])

And "fancy indexing": using an array of indexes to filter and potentially reorder an array:

In [33]:
indexes = numpy.array([7, 5, 1, 0])
small_array[indexes]

array([1007, 1005, 1001, 1000])

In [34]:
small_array[indexes] = 999
small_array

array([         999,          999, 528280977410,            3,
                  4,          999,            6,          999,
                  8,            9])

As in C/C++, you have to be very careful about what returns a view versus what returns a copy:

   * **view:** the new array is a pointer to the same data as the old array; it's faster (does not scale with the size of the array) and changes to the new array affect the old array. There are times when you want that; times when you don't.
   * **copy:** the new array is detached from the old; it's slower to create (sometimes insignificant), and changes to the new array have no effect on the old array.

The `base` attribute of a view is a reference to the array the view views.

In [38]:
view = small_array[4:-2]
copy = small_array[4:-2].copy()

print(view.base)

[         999          999 528280977410            3            4
          999            6          999            8            9]


Let's apply vectorized functions and fancy indexing to a real physics problem. Suppose that you're given an array of `Jet_pt`, an array of `Jet_eta`, and an array of indexes in which each event starts and stops:

In [39]:
import uproot
tree = uproot.open("~/data/NanoAOD-DYJetsToLL.root")["Events"]
pt, eta = tree.arrays(["Jet_pt", "Jet_eta"], outputtype=tuple)
starts, stops = pt.starts, pt.stops
pt = numpy.array(pt)
eta = numpy.array(eta)

In [40]:
print(starts)   # the first event has no jets because starts[0] == stops[0]
print(stops)
print(pt)       # pt[0:5] are for jets in the second event
print(eta)      # eta[0:5] are for jets in the second event, etc.

[      0       0       5 ... 7388387 7388390 7388396]
[      0       5       9 ... 7388390 7388396 7388405]
[29.96875  21.3125   19.671875 ... 16.78125  16.28125  16.25    ]
[-1.880127   3.0981445 -2.9746094 ... -3.5615234  4.625      0.5806885]


**Question 1:** How do we find the events with at least one jet?

In [ ]:
hasajet = ???                   # want array of booleans: tell me what to type!
hasajet

**Question 2:** How do we compute `pz = pt*sinh(eta)` for the first jet in each event?

In [ ]:
indexes = ???                   # want array of integers: tell me what to type!
pz = ???                        # want array of floats: tell me what to type!
pz

Most scientific libraries for Python do the number-crunching in C/C++ and the interface in Python. (One tends to see problems separated into "slow control" and "fast math.") Numpy is the common language that makes it possible to move data from one to another.

Even if a software package isn't Numpy-native but does know about C arrays, you can work with it.

In [41]:
import ROOT
ROOT.gInterpreter.Declare("""
void computemass(int n, float pt1, pt2, eta1, eta2, phi1, phi2, out) {
    TLorentzVector one, two;
    for (int i = 0;  i < n;  i++) {
        one.SetPtEtaPhiM(pt1[i], eta1[i], phi1[i], 0.1056583745);
        two.SetPtEtaPhiM(pt2[i], eta2[i], phi2[i], 0.1056583745);
        out[i] = (one + two).M();
    }
}""")

ImportError: No module named ROOT

In [42]:
2 + 2

4